In [40]:
import matplotlib.pyplot as plt
import numpy as np

def plot_mixer_quant(run_collector, title, key="ReturnAverage", max_n=None, labels=None, xmax=None, colors=None, 
                     lines=None):
    """
    Plot quantiles over a number of runs
    """
    train = run_collector[key]

    for i, key in enumerate(train.keys()):
        runs = train[key]
        train_quantiles = get_quant_stats(runs, max_n=max_n, xmax=xmax)

        train_x = range(len(train_quantiles[1]))

        train_q1 = np.array(train_quantiles[0])
        train_qmean = np.array(train_quantiles[1])
        train_q2 = np.array(train_quantiles[2])
        

        qmean = train_qmean
        q1_offset = qmean - train_q1
        q2_offset = train_q2 - qmean

        plt.fill_between(train_x, qmean-q1_offset, qmean+q2_offset, alpha=0.2, color=colors[i])
        plt.plot(train_x, qmean, color=colors[i], label=labels[i], linestyle=lines[i])
        

    plt.ylabel("Average Return")
    plt.xlabel("Time steps (x 500) ")
    plt.legend(loc='lower right')
    plt.grid(True, linewidth=0.25)
    plt.title(title)
    plt.show()

In [41]:
from scipy.stats import mstats

def get_quant_stats(runs, max_n=None, xmax=None):
    """
    Get 25%, 50% and 75% quantiles
    """
    if xmax is None:
        runs = [x for x in runs if len(x) == 201]
    else:
        runs = [x[:xmax] for x in runs if len(x) >= xmax ]
    runs = np.concatenate([np.expand_dims(np.array(x),0) for x in runs], axis=0)
    if max_n is not None:
        runs = runs[:max_n]
    quantiles = mstats.mquantiles(runs, axis=0)
    return quantiles

In [42]:
from helpers import Folder, get_run_collector

# Name of folder that logs are stored in and the lable
folders = []
labels = []
for const in [0.01, 0.015, 0.02, 0.03, 1.00]:
    folders.append(f"CartpoleExperiment_equivariant_{const}")
    labels.append(f"{const}")
# Plot colors
colors = ["#009999", "#FFA500", "#FF3333", "#800080", "#3d3d3d"] #, "#616161"]
# Plot linestyles
lines = ["solid", "solid", "solid", "solid", "solid"] #], "solid"]

# Every run is stored in a top level folder named for the date it was started on. Replace this list with the 
# correct date/folder names
top_folders = ["20240206", "20240207"]

# Can also replace this with one of the other values logged during training, e.g. ReturnMax or ReturnMedian
values = ["ReturnAverage"]
# If this list is empty, all runs regardless of setting are aggregated. 
# If you are tuning learning rates, split on "lr" as well
split_on = ["const"]

# Collects the relevant logs
run_collector = get_run_collector(split_on, top_folders, folders, values)

# Plot wrapper
plot_mixer_quant(run_collector, title="", key="ReturnAverage", labels=labels, xmax=150, colors=colors, lines=lines)

IndexError: list index out of range